In [24]:
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

# SOT(SOURCE OF TRUTH)

In [1]:
#SOT
import requests
import urllib

def getSOTResults(query):
    channelId="UCAuUUnT6oDeKwE6v1NGQxug"  
    key={apikey} #add api key here
    url="https://content-youtube.googleapis.com/youtube/v3/search?maxResults=100&channelId="+channelId+"&part=snippet&q="+urllib.parse.quote(query)+"&key="+key
    referer="https://content-youtube.googleapis.com/static/proxy.html?usegapi=1&jsh=m%3B%2F_%2Fscs%2Fapps-static%2F_%2Fjs%2Fk%3Doz.gapi.en.g8agzr_oroM.O%2Fam%3DAQ%2Fd%3D1%2Frs%3DAGLTcCP6z3gW3iZ5SpDBmGgDQznnZEz5gQ%2Fm%3D__features__"
    x = requests.get(url, headers={"referer":"temp"})
    res=x.json()
    resArr=[]
    for item in res["items"]:
        if("id" in item and "videoId" in item["id"]):
            resArr.append(item["id"]["videoId"])
            print(item["snippet"]["title"])
            #print("\n")
    return resArr

In [2]:
getSOTResults("AI can bring 2nd rev")

How AI can bring on a second Industrial Revolution | Kevin Kelly
How AI is making it easier to diagnose disease | Pratik Shah
No one should die because they live too far from a doctor | Raj Panjabi
Shai Agassi: A new ecosystem for electric cars
Anil Gupta: India&#39;s hidden hotbeds of invention
Will automation take away all our jobs? | David Autor
Ricardo Semler: Radical wisdom for a company, a school, a life
The walk from &quot;no&quot; to &quot;yes&quot; | William Ury
What I learned as a prisoner in North Korea | Euna Lee
What is so special about the human brain? | Suzana Herculano-Houzel
Why fascism is so tempting -- and how your data could power it | Yuval Noah Harari
Small rockets are the next space revolution | Peter Beck
How India could pull off the world&#39;s most ambitious energy transition | Varun Sivaram
Earth&#39;s mass extinctions | Peter Ward
How the pandemic will shape the near future | Bill Gates
Mentalism, mind reading and the art of getting inside your head | Derren

['IjbTiRbeNpM',
 'mhEYvrFOP88',
 'N9HF8mMe2pU',
 'FcoJt2KLC9k',
 'JHk9YVjhk7c',
 'th3nnEpITz0',
 'k4vzhweOefs',
 'Hc6yi_FtoNo',
 'ZL-6MeYBeUs',
 '_7_XH1CBzGw',
 'xHHb7R3kx40',
 'DhnBn_c9f8Q',
 'Pgq_CODucg0',
 '3lYN_lXU9PA',
 'jmQWOPDqxWA',
 '2IFa0tqHrwE',
 'AO4In7d6X-c',
 'x4m6j7rSyzY',
 'M4vqr3_ROIk',
 'R1vskiVDwl4',
 'tiwVMrTLUWg',
 'CVa_IZVzUoc',
 'nUdsTizSxSI',
 'GOCUH7TxHRI',
 'BTB_HPL2r88',
 'jATCr-gQvPA',
 'IRVdiHu1VCc',
 'iB4MS1hsWXU',
 'CyGWML6cI_k',
 'QpmsftF2We4',
 'xRb7_ffl2D0',
 'HiwJ0hNl1Fw',
 '2cYDyMnL4M8',
 'GcJxRqTs5nk',
 '5aH2Ppjpcho',
 '0zvrGiPkVcs',
 'tH5iEf9oxaI',
 'avuhY7D71sQ',
 'Eg_ToU7m1MI',
 'jM9Q3xP2iBo',
 '2m9kC1yRnLQ',
 'UCKfvxnljYY',
 'rfi3w9Bzwik',
 'UMhLBPPtlrY',
 '5qaktKVL_zM',
 'CEN4XNth61o',
 'ngFXRh3ahm8',
 'ONs9FCY74p0',
 'r92jUj7gNRw']

# SOLR+neuralRerank

In [3]:
import numpy as np
import pandas as pd
import transformers
import gensim
import re
import nltk
import random
from nltk.stem import WordNetLemmatizer
sot_relevant_res=[3,4,5]
rand = random.Random(500)
stemmer = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

ft_model = gensim.models.Word2Vec.load("ft3.model")
embed_size = {embeddingSize} #add here

def l2_norm(x):
    return np.sqrt(np.sum(x**2))

def div_norm(x):
    norm_value = l2_norm(x)
    if norm_value > 0:
        return x * ( 1.0 / norm_value)
    else:
        return x
    
def getSentenceEmbedding(sent, model):
    sentArr=sent.split(" ")
    start = np.zeros(embed_size)
    for word in sentArr:
        word_embed=model.wv[word]
        start+=div_norm(word_embed)
    return start/len(sentArr)

def parse_text(document):
        document = re.sub(r'\W', ' ', str(document))
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        document = re.sub(r'^b\s+', '', document)
        document = document.lower()
        tokens = document.split()
        tokens = [stemmer.lemmatize(word) for word in tokens]
        tokens = [word for word in tokens if word not in en_stop]
        tokens = [word for word in tokens if len(word) > 3]
        preprocessed_text = ' '.join(tokens)
        return preprocessed_text

from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('paraphrase-mpnet-base-v2')
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
# model = SentenceTransformer('msmarco-distilbert-base-dot-prod-v3')
model = SentenceTransformer('msmarco-distilbert-base-v4')
from scipy import spatial

def getFasttextScore(sent1,query):
    dataSetI = getSentenceEmbedding(sent1,ft_model)
    dataSetII = getSentenceEmbedding(query,ft_model)
    result = 1 - spatial.distance.cosine(dataSetI, dataSetII)
    return result

def getBertScore(sent1,query):
    embeddings1 = model.encode([sent1], convert_to_tensor=True)
    embeddings2 = model.encode([query], convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(embeddings1, embeddings2)
    return cosine_scores[0][0]

def getNeuralScore(sent1,query):
    ft_score=getFasttextScore(sent1,query)
    #print("Fastext score: "+str(ft_score))
    bert_score=getBertScore(sent1,query).item()
    #print("Bert score: "+str(bert_score)+"\n")
    return max(0.5*bert_score+0.7*ft_score,max(bert_score,ft_score))

C:\Users\user\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import random

def luceneSer(q):
    url="http://localhost:8983/solr/ljmu/select?defType=dismax&q="+urllib.parse.quote(q)+"&qf=title%5E10%20summary%5E2%20data%5E1&rows=30"
    x= requests.get(url)
    res= x.json()
    #print(res["response"]["numFound"])
    #print(len(res["response"]["docs"]))
    #print()
    return res


def getNeuralOp(q):
    q=parse_text(q)
    res=luceneSer(q)
    op=res["response"]["docs"]
    ids=[]
    if(len(q.split(" "))<3):
        for val in op:
            ids.append(val["id"])
            print(val["title"][0])
        print("Entered here #1")
        return ids
    neuralOp=[]
    for doc in op:
        if "summary" in doc:
            #increase neural weightage of title to 3x of BERT summary
            titleVid=parse_text(doc["title"][0]+" "+doc["title"][0]+" "+doc["title"][0])
            val=titleVid+" "+parse_text(doc["summary"][0])
            score=getNeuralScore(val,q)
            if(score>0.5):
                neuralOp.append({
                    "id":doc["id"],
                    "score":score,
                    "title":doc["title"][0]
                })
    neuralOp.sort(key=lambda elem:elem["score"], reverse=True)
    for val in neuralOp:
        ids.append(val["id"])
        print(val["title"])
    print("Entered here #2")
    return ids

In [5]:
#getNeuralOp("Batman is the best superhero ever and superman is not cool")
getNeuralOp("Batman")

The function and fashion of eyeglasses | Small Thing Big Idea, a TED series
Naif Al-Mutawa: Superheroes inspired by Islam
My journey from Marine to actor | Adam Driver
Dennis Hong: My 7 species of robot
Why it's so hard to make healthy decisions | David Asch
Daniel Kish: How I use sonar to navigate the world
Ze Frank's web playroom
How craving attention makes you less creative | Joseph Gordon-Levitt
How to make work-life balance work | Nigel Marsh
Fred Swaniker: The leaders who ruined Africa, and the generation who can fix it
Lemn Sissay: A child of the state
Richard Resnick: Welcome to the genomic revolution
See invisible motion, hear silent sounds. Cool? Creepy? We can't decide | Michael Rubinstein
How synthetic biology could wipe out humanity -- and how we can stop it | Rob Reid
Close-up card magic with a twist | Lennart Green
Entered here #1


['eeVCz-9SUc8',
 '_K095wuE_eE',
 'nCwwVjPNloY',
 'cfR6tiSW-AM',
 'PI5V1-IFvlI',
 'uH0aihGWB8U',
 '3gSSNHO1dDs',
 '3VTsIju1dLI',
 'jdpIKXLLYYM',
 'kcEIsbO0ivA',
 'sLiM2-izFl4',
 'u8bsCiq6hvM',
 'fHfhorJnAEI',
 '8DDgHq9ewOo',
 '1_oa8m5Oq00']

In [6]:
getNeuralOp("Batman is the best superhero ever and superman is not cool")

The story of Marvel's first queer Latina superhero | Gabby Rivera
The best kindergarten you’ve ever seen | Takaharu Tezuka
Never, ever give up | Diana Nyad
Stacey Kramer: The best gift I ever survived
Morgan Spurlock: The greatest TED Talk ever sold
The best career path isn't always a straight line | Sarah Ellis and Helen Tupper
David Pogue: Cool new things you can do with your mobile
The most detailed map of galaxies, black holes and stars ever made | Juna Kollmeier
Maybe the best robot demo ever | Marco Tempest
Every piece of art you've ever wanted to see -- up close and searchable | Amit Sood
How we can use the hiring process to bring out the best in people | The Way We Work, a TED series
Should we create a solar shade to cool the earth? | Danny Hillis
The best way to help is often just to listen | Sophie Andrews
James Patten: The best computer interface? Maybe ... your hands
Why winning doesn't always equal success | Valorie Kondos Field
Paul Bloom: Can prejudice ever be a good thi

['UeGLnUqNI0s',
 'J5jwEyDaR-0',
 'Zx8uYIfUvh4',
 'PKbet4RdSo4',
 '6c0VtOdibcI',
 '1ALfKWG2nmw',
 'mJxA8eIWQUc',
 '88d-58tWhGs',
 'FJJe8PXEUhk',
 'CjB6DQGalU0',
 '6UiU99_tE7I',
 'zA6_pcEz8Ls',
 'u1_dYypojbE',
 '5WI6ZqCUNk0',
 'JJyeKiT8g4g',
 'KDBcoRLkut8',
 'fHfhorJnAEI',
 'VU44eEKtcmQ',
 '949yOxz_x80',
 'M6TMSQWI9m4',
 'jiDQDLnEXdA',
 'HXbsVbFAczg',
 'P2AUat93a8Q',
 'B6lBtiQZSho',
 'ORthzIOEf30',
 'IwPOPL-ieD0',
 '18r6nchs91E',
 'XguYZXUChhY',
 'ihoR9B7p-1Q',
 'RUwS1uAdUcI']

# MATH PLAYGROUND

In [7]:
from querySentEval import evalP30Score,evalP10Score,evalAPScore
BUCKET_SIZE=0.1
query="AI can bring on a second Industrial Revolution"
evalP30Score(getNeuralOp(query),getSOTResults(query),BUCKET_SIZE)

The next manufacturing revolution is here | Olivier Scalabre
How AI can bring on a second Industrial Revolution | Kevin Kelly
Why "biofabrication" is the next industrial revolution | Suzanne Lee
Imaging at a trillion frames per second | Ramesh Raskar
Marco Annunziata: Welcome to the age of the industrial internet
The next global agricultural revolution | Bruce Friedrich
Eric Dishman: Take health care off the mainframe
Wael Ghonim: Inside the Egyptian revolution
What's needed to bring the US voting system into the 21st century | Tiana Epps-Johnson
Philip Rosedale: Second Life, where anything is possible
Richard Resnick: Welcome to the genomic revolution
Simon Lewis: Don't take consciousness for granted
Nilofer Merchant: Got a meeting? Take a walk
Cesar Kuriyama: One second every day
How we take back the internet | Edward Snowden
Bring on the learning revolution! | Ken Robinson
Bring on the learning revolution! | Sir Ken Robinson
Rebecca MacKinnon: Let's take back the Internet!
Me Too is

0.55

# EVAL Metrics

### TrecMB15

In [26]:
#MS-marco P30
filepath = "query/mb15-fin.txt"
qs=[] #query set
with open(filepath,encoding="utf8") as fp:
    for cnt, line in enumerate(fp):
        qs.append(line)
qs = random.sample(qs, 200)
p30QuerySetMean=0
cnt=1
docsFnd=0
for ele in qs:
    ele=parse_text(ele)
    sotArr=getSOTResults(ele)
    if(len(sotArr)>0):
        p30QuerySetMean+=evalP30Score(getNeuralOp(ele),sotArr,BUCKET_SIZE)
        docsFnd+=1
    cnt+=1
p30QuerySetMean=p30QuerySetMean/docsFnd

Thomas Insel: Toward a new understanding of mental illness
How childhood trauma affects health across a lifetime | Nadine Burke Harris
Do schools kill creativity? | Sir Ken Robinson
David Anderson: Your brain is more than a bag of chemicals
Ali Carr-Chellman: Gaming to re-engage boys in learning
Read Montague: What we&#39;re learning from 5,000 brains
This is your brain on air pollution | María Neira
What happens in your brain when you pay attention? | Mehdi Ordikhani-Seyedlar
There's no shame in taking care of your mental health | Sangu Delle
The hidden role informal caregivers play in health care | Scott Williams
Why your doctor should care about social justice | Mary Bassett
How to practice emotional first aid | Guy Winch
What the US health care system assumes about you | Mitchell Katz
Nicolas Perony: Puppies! Now that I've got your attention, complexity theory
When do kids start to care about other people's opinions? | Sara Valencia Botto
Thulasiraj Ravilla: How low-cost eye care c

What will a future without secrets look like? | Alessandro Acquisti
10 things you didn&#39;t know about orgasm | Mary Roach
What should electric cars sound like? | Renzo Vitale
Malte Spitz: Your phone company is watching
It&#39;s OK to feel overwhelmed. Here&#39;s what to do next | Elizabeth Gilbert
The search for our solar system&#39;s ninth planet | Mike Brown
Hacking your memory -- with sleep | Sleeping with Science, a TED series
Design at the Intersection of Technology and Biology | Neri Oxman | TED Talks
How the pandemic will shape the near future | Bill Gates
Why does the universe exist? | Jim Holt
This could be why you&#39;re depressed or anxious | Johann Hari
When technology can read minds, how will we protect our privacy? | Nita Farahany
How trees talk to each other | Suzanne Simard
What happens when our computers get smarter than we are? | Nick Bostrom
How to build your confidence -- and spark it in others | Brittany Packnett
Beware online &quot;filter bubbles&quot; | Eli Par

How to spot a liar | Pamela Meyer
The happy secret to better work | Shawn Achor
10 things you didn&#39;t know about orgasm | Mary Roach
How the blockchain is changing money and business | Don Tapscott
Secrets of success in 8 words, 3 minutes | Richard St. John
&quot;Everything happens for a reason&quot; -- and other lies I&#39;ve loved | Kate Bowler
Your brain on video games | Daphne Bavelier
Connected, but alone? | Sherry Turkle
What happens when our computers get smarter than we are? | Nick Bostrom
A beginner&#39;s guide to quantum computing | Shohini Ghose
What Islam really says about women | Alaa Murabit
How to escape education&#39;s death valley | Sir Ken Robinson
What you don&#39;t know about marriage | Jenna McCarthy
Why 30 is not the new 20 | Meg Jay
How we take back the internet | Edward Snowden
The incredible inventions of intuitive AI | Maurice Conti
Peter Donnelly: How stats fool juries
We don&#39;t &quot;move on&quot; from grief. We move forward with it | Nora McInerny
A S

Everything you think you know about addiction is wrong | Johann Hari
An interview with the founders of Black Lives Matter | Alicia Garza, Patrisse Cullors, Opal Tometi
Leana Wen: What your doctor won’t disclose
Why your doctor should care about social justice | Mary Bassett
A little-told tale of sex and sensuality | Shereen El Feki
How digital innovation can fight pandemics and strengthen democracy | Audrey Tang
Love, no matter what | Andrew Solomon
What doctors should know about gender identity | Kristie Overstreet
How judges can show respect | Victoria Pratt
Norman Spack: How I help transgender teens become who they want to be
Del Harvey: The strangeness of scale at Twitter
Why kids need to learn about gender and sexuality | Lindsay Amer
A bold idea to replace politicians | César Hidalgo
Language around gender and identity evolves (and always has) | Archie Crowley
The secret to great opportunities? The person you haven't met yet | Tanya Menon
Andy Yen: Think your email's private? Thi

The secret to giving great feedback | The Way We Work, a TED series
How the worst moments in our lives make us who we are | Andrew Solomon
The role of faith and belief in modern Africa | Ndidi Nwuneli
Keith Chen: Could your language affect your ability to save money?
How childhood trauma affects health across a lifetime | Nadine Burke Harris
Verna Myers: How to overcome our biases? Walk boldly toward them
How to be your best self in times of crisis | Susan David
Robyn Stein DeLuca: The good news about PMS
Baba Shiv: Sometimes it's good to give up the driver's seat
What's so sexy about math? | Cédric Villani
3 ways to practice civility | Steven Petrow
Jonathan Haidt: Religion, evolution, and the ecstasy of self-transcendence
How couples can sustain a strong sexual connection for a lifetime | Emily Nagoski
Verna Myers: How to overcome our biases? Walk boldly toward them
When do kids start to care about other people's opinions? | Sara Valencia Botto
6 essential lessons for women leaders |

Looking for a job? Highlight your ability, not your experience | Jason Shen
A lesson on looking | Amy Herman
Looks aren&#39;t everything. Believe me, I&#39;m a model. | Cameron Russell
Caroline Casey: Looking past limits
Magical houses, made of bamboo | Elora Hardy
The violin, and my dark night of the soul | Ji-Hae Park
What Islam really says about women | Alaa Murabit
Mind-blowing, magnified portraits of insects | Levon Biss
How I became an entrepreneur at 66 | Paul Tasner
What happens when you have a disease doctors can&#39;t diagnose | Jennifer Brea
Asking for help is a strength, not a weakness | Michele L. Sullivan
10 ways to have a better conversation | Celeste Headlee
Daniel Goldstein: The battle between your present and future self
Lessons from the longest study on human development | Helen Pearson
The hidden power of smiling | Ron Gutman
Shekhar Kapur: We are the stories we tell ourselves
Building a dinosaur from a chicken | Jack Horner
Should you be able to patent a human gene

The secret to giving great feedback | The Way We Work, a TED series
TED&#39;s secret to great public speaking | Chris Anderson
The business benefits of doing good | Wendy Woods
The single biggest reason why start-ups succeed | Bill Gross
Looking for a job? Highlight your ability, not your experience | Jason Shen
Why the secret to success is setting the right goals | John Doerr
How language shapes the way we think | Lera Boroditsky
The violin, and my dark night of the soul | Ji-Hae Park
How to raise successful kids -- without over-parenting | Julie Lythcott-Haims
How great leaders inspire action | Simon Sinek
Liz Diller: Architecture is a special effects machine
Michael Pollan: A plant&#39;s-eye view
Every kid needs a champion | Rita Pierson
How to fix a broken heart | Guy Winch
The opportunity of adversity | Aimee Mullins
How AI is making it easier to diagnose disease | Pratik Shah
Shaffi Mather: A new way to fight corruption
Ben Cameron: The true power of the performing arts
What your

In [27]:
print("P30 Microblog 2015 score: " + str(p30QuerySetMean))

P30 Microblog 2015 score: 0.4916666666666667


In [8]:
#MS-marco P10
filepath = "query/mb15-fin.txt"
qs=[] #query set
with open(filepath,encoding="utf8") as fp:
    for cnt, line in enumerate(fp):
        qs.append(line)
qs = random.sample(qs, 200)
p10QuerySetMean=0
cnt=1
docsFnd=0
for ele in qs:
    ele=parse_text(ele)
    sotArr=getSOTResults(ele)
    if(len(sotArr)>0):
        p10QuerySetMean+=evalP10Score(getNeuralOp(ele),sotArr,BUCKET_SIZE)
        docsFnd+=1
    cnt+=1
p10QuerySetMean=p10QuerySetMean/docsFnd

The paradox of efficiency | Edward Tenner
3 steps to stop remote work burnout | The Way We Work, a TED series
How Too Many Rules at Work Keep You from Getting Things Done | Yves Morieux | TED Talks
Are indoor vertical farms the future of agriculture? | Stuart Oda
7 principles for building better cities | Peter Calthorpe
The harm reduction model of drug addiction treatment | Mark Tyndall
A global pandemic calls for global solutions | Larry Brilliant
Learning from dirty jobs | Mike Rowe
The danger of AI is weirder than you think | Janelle Shane
100 solutions to reverse global warming | Chad Frischmann
Eric Giler demos wireless electricity
Amory Lovins: We must win the oil endgame
Sendhil Mullainathan: Solving social problems with a nudge
Sleep is your superpower | Matt Walker
How fear of nuclear power is hurting the environment | Michael Shellenberger
Innovating to zero! | Bill Gates
How to decarbonize the grid and electrify everything | John Doerr and Hal Harvey
A printable, flexible, o

How language shapes the way we think | Lera Boroditsky
How We Can Make the World a Better Place by 2030 | Michael Green | TED Talks
Ben Wellington: How we found the worst place to park in New York City — using big data
Reviving New York&#39;s rivers -- with oysters! | Kate Orff
Why domestic violence victims don&#39;t leave | Leslie Morgan Steiner
The state of the climate -- and what we might do about it | Nicholas Stern
The mathematician who cracked Wall Street | Jim Simons
The ancient, earth-friendly wisdom of Mongolian nomads | Khulan Batkhuyag
5 needs that any COVID-19 response should meet | Kwame Owusu-Kesse
How American and Chinese values shaped the coronavirus response | Huang Hung
The riddle of experience vs. memory | Daniel Kahneman
How one tweet can ruin your life | Jon Ronson
John Maeda: Designing for simplicity
The psychology of evil | Philip Zimbardo
What tech companies know about your kids | Veronica Barassi
7 principles for building better cities | Peter Calthorpe
Looking

Paul Bloom: Can prejudice ever be a good thing?
Looking for a job? Highlight your ability, not your experience | Jason Shen
Your brain hallucinates your conscious reality | Anil Seth
Ze Frank&#39;s web playroom
How can groups make good decisions? | Mariano Sigman and Dan Ariely
The first secret of great design | Tony Fadell
The danger of a single story | Chimamanda Ngozi Adichie
How Amazon, Apple, Facebook and Google manipulate our emotions | Scott Galloway
Kevin Kelly: The next 5,000 days of the web
My stroke of insight | Jill Bolte Taylor
The future we&#39;re building -- and boring | Elon Musk
How a new species of ancestors is changing our theory of human evolution | Juliet Brophy
What&#39;s needed to bring the US voting system into the 21st century | Tiana Epps-Johnson
A 12-year-old app developer | Thomas Suarez
Gary Flake: is Pivot a turning point for web exploration?
Beware online &quot;filter bubbles&quot; | Eli Pariser
Your body language may shape who you are | Amy Cuddy
The fut

10 things you didn&#39;t know about orgasm | Mary Roach
What really matters at the end of life | BJ Miller
Michael Anti: Behind the Great Firewall of China
Understanding the rise of China | Martin Jacques
Thoughts on humanity, fame and love | Shah Rukh Khan
Kiran Bedi: How I remade one of India&#39;s toughest prisons
The biggest risks facing cities -- and some solutions | Robert Muggah
Jonathan Zittrain: The Web as random acts of kindness
Ellen Dunham-Jones: Retrofitting suburbia
Three ideas. Three contradictions. Or not. | Hannah Gadsby
Michael Pollan: A plant's-eye view
We need to track the world's water like we track the weather | Sonaar Luthra
Lian Pin Koh: A drone's-eye view of conservation
Aditi Shankardass: A second opinion on learning disorders
The revolutionary power of diverse thought | Elif Shafak
Let's launch a satellite to track a threatening greenhouse gas | Fred Krupp
Vincent Cochetel: I was held hostage for 317 days. Here's what I thought about…
Catherine Crump: The sma

The genius behind some of the world&#39;s most famous buildings | Renzo Piano
Why great architecture should tell a story | Ole Scheeren
Teddy Cruz: How architectural innovations migrate across borders
Why good leaders make you feel safe | Simon Sinek
Afra Raymond: Three myths about corruption
The future we&#39;re building -- and boring | Elon Musk
Hasan Elahi: FBI, here I am!
Our campaign to ban plastic bags in Bali | Melati and Isabel Wijsen
Suzanne Talhouk: Don&#39;t kill your language
The unexpected challenges of a country&#39;s first election | Philippa Neave
Why I put myself in danger to tell the stories of Gaza | Ameera Harouda
David Binder: The arts festival revolution
Why I keep speaking up, even when people mock my accent | Safwat Saleem
Why you should make useless things | Simone Giertz
How we teach computers to understand pictures | Fei Fei Li
Why 30 is not the new 20 | Meg Jay
How one tweet can ruin your life | Jon Ronson
Google&#39;s driverless car | Sebastian Thrun
Michae

I got 99 problems... palsy is just one | Maysoon Zayid
What does it mean to be a citizen of the world? | Hugh Evans
Why humans run the world | Yuval Noah Harari
Roxane Gay: Confessions of a bad feminist
David Binder: The arts festival revolution
Can a computer write poetry? | Oscar Schwartz
Poetry, music and identity (with English subtitles) | Jorge Drexler
How New Technology Helps Blind People Explore the World | Chieko Asakawa | TED Talks
Street Art for Hope and Peace | eL Seed | TED Talks
Jeff Smith: Lessons in business ... from prison
Poetry that frees the soul (with English subtitles) | Cristina Domenech
Billy Collins: Everyday moments, caught in time
Lies, damned lies and statistics (about TEDTalks)
4 Powerful Poems about Parkinson's and Growing Older | Robin Morgan | TED Talks
Del Harvey: The strangeness of scale at Twitter
McKenna Pope: Want to be an activist? Start with your toys
The most powerful woman you've never heard of | T. Morgan Dixon and Vanessa Garrison
Enough with t

How racism makes us sick | David R. Williams
Why Gender Equality Is Good for Everyone — Men Included | Michael Kimmel | TED Talks
Gary Wolf: The quantified self
Daniel Goldstein: The battle between your present and future self
Shimon Schocken: The self-organizing computer course
The psychology of your future self | Dan Gilbert
Jonathan Haidt: Religion, evolution, and the ecstasy of self-transcendence
What will future jobs look like? | Andrew McAfee
What you need to know about face surveillance | Kade Crockford
Chris Anderson: How YouTube is driving innovation
Rana el Kaliouby: This app knows how you feel — from the look on your face
What makes life worth living in the face of death | Lucy Kalanithi
What a driverless world could look like | Wanis Kabbaj
Hod Lipson: Robots that are "self-aware"
What makes us get sick? Look upstream | Rishi Manchanda
The pattern behind self-deception | Michael Shermer
An honest look at the personal finance crisis | Elizabeth White
How we can face the futu

In [9]:
print("P10 Microblog 2015 score: " + str(p10QuerySetMean))

P10 Microblog 2015 score: 0.4009259259259259


In [10]:
#MS-marco AP
filepath = "query/mb15-fin.txt"
qs=[] #query set
with open(filepath,encoding="utf8") as fp:
    for cnt, line in enumerate(fp):
        qs.append(line)
qs = random.sample(qs, 200)
apQuerySetMean=0
cnt=1
docsFnd=0
for ele in qs:
    ele=parse_text(ele)
    sotArr=getSOTResults(ele)
    if(len(sotArr)>0):
        apQuerySetMean+=evalP10Score(getNeuralOp(ele),sotArr,BUCKET_SIZE)
        docsFnd+=1
        print("Eval Count: "+str(cnt) + "==========================>")
    cnt+=1
apQuerySetMean=apQuerySetMean/docsFnd

The secret to giving great feedback | The Way We Work, a TED series
The three ways that good design makes you happy | Don Norman
How language shapes the way we think | Lera Boroditsky
How a handful of tech companies control billions of minds every day | Tristan Harris
The danger of silence | Clint Smith
How to speak so that people want to listen | Julian Treasure
The magic of Fibonacci numbers | Arthur Benjamin
Let&#39;s teach for mastery -- not test scores | Sal Khan
Why the secret to success is setting the right goals | John Doerr
Grit: the power of passion and perseverance | Angela Lee Duckworth
Tal Danino: We can use bacteria to detect cancer (and maybe treat it)
Why 30 is not the new 20 | Meg Jay
TED&#39;s secret to great public speaking | Chris Anderson
Why humans run the world | Yuval Noah Harari
Jeff Hancock: 3 types of (digital) lies
The way we think about charity is dead wrong | Dan Pallotta
The urgency of intersectionality | Kimberlé Crenshaw
What you should know about vapin

The orchestra in my mouth | Tom Thum
Reggie Watts disorients you in the most entertaining way
How changing your story can change your life | Lori Gottlieb
Anil Gupta: India&#39;s hidden hotbeds of invention
Bran Ferren: To create for the ages, let&#39;s combine art and engineering
How I use the drum to tell my story | Kasiva Mutua
Ballroom dance that breaks gender roles | Trevor Copp and Jeff Fox
Why we must stop dancing to the sound of our own oppression | Madame Gandhi
Dancing with light | Quixotic Fusion
Kartick Satyanarayan: How we rescued the "dancing" bears
What soccer can teach us about freedom | Marc Bamuthi Joseph
Péter Fankhauser: Meet Rezero, the dancing ballbot
Adam Sadowsky: How to engineer a viral music video
How video game skills can get you ahead in life | William Collis
A video game to cope with grief | Amy Green
A dance to honor Mother Earth | Jon Boogz and Lil Buck
Steve Keil: A manifesto for play, for Bulgaria and beyond
John Hunter: Teaching with the World Peace Ga

Thomas Goetz: It's time to redesign medical data
Kenneth Cukier: Big data is better data
Where's Google going next? | Larry Page
When you're making a deal, what's going on in your brain? | Colin Camerer
How work kept me going during my cancer treatment | Sarah Donnelly
How to find a wonderful idea | OK Go
How we can find ourselves in data | Giorgia Lupi
Rory Sutherland: Sweat the small stuff
Paul Kemp-Robertson: Bitcoin. Sweat. Tide. Meet the future of branded currency.
How the mysterious dark net is going mainstream | Jamie Bartlett
How to find work you love | Scott Dinsmore
David Carson: Design, discovery and humor
How to see past your own perspective and find truth | Michael Patrick Lynch
Success, failure and the drive to keep creating | Elizabeth Gilbert
Robert Full: Secrets of movement, from geckos and roaches
What I learned from going blind in space | Chris Hadfield
Me Too is a movement, not a moment | Tarana Burke
Philip Evans: How data will transform business
Anna Mracek Dietri

The world's largest family reunion ... we're all invited! | A.J. Jacobs
How to use family dinner to teach politics | Hajer Sharief
Family, hope and resilience on the migrant trail | Jon Lowenstein
Bruce Feiler: Agile programming -- for your family
How we're building the world's largest family tree | Yaniv Erlich
Dilip Ratha: The hidden force in global economics: sending money home
My $500 house in Detroit -- and the neighbors who helped me rebuild it | Drew Philp
Enrique Peñalosa: Why buses represent democracy in action
How to turn your dissatisfaction into action | Yvonne Aki-Sawyerr
How great leaders inspire action | Simon Sinek
Bono: Action for Africa
Award-winning teen-age science in action
Pico Iyer: Where is home?
Do kids think of sperm donors as family? | Veerle Provoost
The economic benefits of climate action | Marcelo Mena
Love, sorrow and the emotions that power climate action | Knut Ivar Bjørlykhaug
The family I lost in North Korea. And the family I gained | Joseph Kim
Xavie

JP Rangaswami: Information is food
How we can find ourselves in data | Giorgia Lupi
Insightful human portraits made from data | R. Luke DuBois
Andy Yen: Think your email's private? Think again
Anders Ynnerman: Visualizing the medical data explosion
Your fingerprints reveal more than you think | Simona Francese
How to discover your "why" in difficult times | Simon Sinek
David Carson: Design, discovery and humor
Kenneth Cukier: Big data is better data
Talithia Williams: Own your body's data
Why you think you're right -- even if you're wrong | Julia Galef
Where joy hides and how to find it | Ingrid Fetell Lee
Help discover ancient ruins -- before it's too late | Sarah Parcak
How to find a wonderful idea | OK Go
Who am I? Think again | Hetain Patel
How to find work you love | Scott Dinsmore
How language shapes the way we think | Lera Boroditsky
Alison Gopnik: What do babies think?
How record collectors find lost music and preserve our cultural heritage | Alexis Charpentier
Ash Beckham: We'

Want to be more creative? Go for a walk | Marily Oppezzo
How to design a library that makes kids want to read |  Michael Bierut
McKenna Pope: Want to be an activist? Start with your toys
Want to get great at something? Get a coach | Atul Gawande
My mother's final wish -- and the right to die with dignity | Elaine Fong
The reporting system that sexual assault survivors want | Jessica Ladd
I don't want children -- stop telling me I'll change my mind | Christen Reighter
How to speak so that people want to listen | Julian Treasure
The secret to desire in a long-term relationship | Esther Perel
Portraits that transform people into whatever they want to be | Uldus Bakhtiozina
The laws that sex workers really want | Juno Mac
Joseph Pine: What consumers want
Want to be happier? Stay in the moment | Matt Killingsworth
Norman Spack: How I help transgender teens become who they want to be
Charmian Gooch: My wish: To launch a new era of openness in business
Dave Eggers: 2008 TED Prize wish: Once U

The art of misdirection | Apollo Robbins
10 ways to have a better conversation | Celeste Headlee
Every kid needs a champion | Rita Pierson
The magic of Fibonacci numbers | Arthur Benjamin
My love letter to cosplay | Adam Savage
How to speak so that people want to listen | Julian Treasure
The power of vulnerability | Brené Brown
A simple way to break a bad habit | Judson Brewer
Get comfortable with being uncomfortable | Luvvie Ajayi Jones
How I climbed a 3,000-foot vertical cliff -- without ropes | Alex Honnold
How to gain control of your free time | Laura Vanderkam
The difference between healthy and unhealthy love | Katie Hood
Why good leaders make you feel safe | Simon Sinek
How books can open your mind | Lisa Bu
Are athletes really getting faster, better, stronger? | David Epstein
Why you should define your fears instead of your goals | Tim Ferriss
A guerilla gardener in South Central LA | Ron Finley
The voices in my head | Eleanor Longden
Learning from dirty jobs | Mike Rowe
Making 

In [11]:
print("AP Microblog 2015 score: " + str(apQuerySetMean))

AP Microblog 2015 score: 0.3638888888888888


## Robust-05

In [12]:
#MS-marco P30
filepath = "query/rob05-fin.txt"
qs=[] #query set
with open(filepath,encoding="utf8") as fp:
    for cnt, line in enumerate(fp):
        qs.append(line)
qs = random.sample(qs, 200)
p30QuerySetMean=0
cnt=1
docsFnd=0
for ele in qs:
    ele=parse_text(ele)
    sotArr=getSOTResults(ele)
    if(len(sotArr)>0):
        p30QuerySetMean+=evalP30Score(getNeuralOp(ele),sotArr,BUCKET_SIZE)
        docsFnd+=1
    cnt+=1
p30QuerySetMean=p30QuerySetMean/docsFnd

A simple way to break a bad habit | Judson Brewer
How to fix a broken education system ... without any more money | Seema Bansal
Math is the hidden secret to understanding the world | Roger Antonsen
Nationalism vs. globalism: the new political divide | Yuval Noah Harari
Can we cure genetic diseases by rewriting DNA? | David R. Liu
The pattern behind self-deception | Michael Shermer
Frederic Kaplan: How I built an information time machine
The quest for the coronavirus vaccine | Seth Berkley
Angela Belcher: Using nature to grow batteries
Rob Knight: How our microbes make us who we are
The secret weapon against pandemics | Georges C. Benjamin
What refugees need to start new lives | Muhammed Idris
George Whitesides: A lab the size of a postage stamp
Mark Kendall: Demo: A needle-free vaccine patch that's safer and way cheaper
Frederic Kaplan: How I built an information time machine
Ray Kurzweil: Get ready for hybrid thinking
What is so special about the human brain? | Suzana Herculano-Houze

Juliana Machado Ferreira: The fight to end rare-animal trafficking in Brazil
Photos that give voice to the animal kingdom | Frans Lanting
Pamela Ronald: The case for engineering our food
Tristram Stuart: The global food waste scandal
Why I study the most dangerous animal on earth -- mosquitoes | Fredros Okumu
Grief and love in the animal kingdom | Barbara J. King
The tiny creature that secretly powers the planet | Penny Chisholm
Birke Baehr: What's wrong with our food system
The future of good food in China | Matilda Ho
Ellen Gustafson: Obesity + hunger = 1 global food issue
Depressed dogs, cats with OCD -- what animal madness means for us humans | Laurel Braitman
A friendly, autonomous robot that delivers your food | Ali Kashani
David Brooks: The social animal
Teach every child about food | Jamie Oliver
JP Rangaswami: Information is food
Carolyn Steel: How food shapes our cities
Sheila Patek: Measuring the fastest animal on earth
How to read the genome and build a human being | Riccar

Your brain on video games | Daphne Bavelier
What it&#39;s like to be a Muslim in America | Dalia Mogahed
Annie Lennox: Why I am an HIV/AIDS activist
We don&#39;t &quot;move on&quot; from grief. We move forward with it | Nora McInerny
Inside the mind of a master procrastinator | Tim Urban
I grew up in the Westboro Baptist Church. Here&#39;s why I left | Megan Phelps-Roper
Yochai Benkler: Open-source economics
Rebecca MacKinnon: Let's take back the Internet!
Andrew Mwenda: Let's take a new look at African aid
Edi Rama: Take back your city with paint
How we take back the internet | Edward Snowden
How to lead a conversation between people who disagree | Eve Pearlman
Matthieu Ricard: How to let altruism be your guide
Will automation take away all our jobs? | David Autor
3 lessons of revolutionary love in a time of rage | Valarie Kaur
Kiran Bir Sethi teaches kids to take charge
Eric Dishman: Take health care off the mainframe
Gary Lauder's new traffic sign: Take Turns
A guide to collaborativ

Marc Abrahams: A science award that makes you laugh, then think
James Watson: How we discovered DNA
How CRISPR lets us edit our DNA | Jennifer Doudna
Why humans run the world | Yuval Noah Harari
Inside the black hole image that made history | Sheperd Doeleman
The science of cells that never get old | Elizabeth Blackburn
The search for dark matter -- and what we&#39;ve found so far | Risa Wechsler
Fake videos of real people -- and how to spot them | Supasorn Suwajanakorn
Why I fight for the education of refugee girls (like me) | Mary Maker
Andrea Ghez: The hunt for a supermassive black hole
Why is our universe fine-tuned for life? | Brian Greene
How to stay calm when you know you&#39;ll be stressed | Daniel Levitin
The danger of AI is weirder than you think | Janelle Shane
Murray Gell-Mann: Do all languages have a common ancestor?
David Cameron: The next age of government
The Great Migration and the power of a single decision | Isabel Wilkerson
Why journalists have an obligation to chal

A precise, three-word address for every place on earth | Chris Sheldrick
Pico Iyer: Where is home?
The single biggest reason why start-ups succeed | Bill Gross
My failed mission to find God -- and what I found instead | Anjali Kumar
My escape from North Korea | Hyeonseo Lee
Rajesh Rao: Computing a Rosetta Stone for the Indus script
How to design a library that makes kids want to read |  Michael Bierut
The urgency of intersectionality | Kimberlé Crenshaw
Brain magic | Keith Barry
What I learned about freedom after escaping North Korea | Yeonmi Park
How to spot a liar | Pamela Meyer
Your brain on video games | Daphne Bavelier
Thoughts on humanity, fame and love | Shah Rukh Khan
Tierney Thys: Swim with giant sunfish in the open ocean
Ellen Dunham-Jones: Retrofitting suburbia
Karen Bass: Unseen footage, untamed nature
A simple way to break a bad habit | Judson Brewer
How to take a picture of a black hole | Katie Bouman
6 tips for better sleep | Sleeping with Science, a TED series
Who are y

The reporting system that sexual assault survivors want | Jessica Ladd
Ziyah Gafić: Everyday objects, tragic histories
Dear Facebook, this is how you're breaking democracy | Yael Eisenstat
Jarreth Merz: Filming democracy in Ghana
How free is our freedom of the press? | Trevor Timm
Academic research is publicly funded -- why isn't it publicly available? | Erica Stone
Stefan Larsson: What doctors can learn from each other
Can we learn to talk to sperm whales? | David Gruber
Sarah Kaminsky: My father the forger
How we can use the hiring process to bring out the best in people | The Way We Work, a TED series
Stunning buildings made from raw, imperfect materials | Débora Mesa Molina
Read Montague: What we're learning from 5,000 brains
The astounding athletic power of quadcopters | Raffaello D'Andrea
3 things new parents should consider before going back to work | The Way We Work, a TED series
Ken Kamler: Medical miracle on Everest
A video game to cope with grief | Amy Green
Need a new idea?

In [13]:
print("P30 Robust-05 score: " + str(p30QuerySetMean))

P30 Robust-05 score: 0.39999999999999997


In [14]:
#MS-marco P10
filepath = "query/rob05-fin.txt"
qs=[] #query set
with open(filepath,encoding="utf8") as fp:
    for cnt, line in enumerate(fp):
        qs.append(line)
qs = random.sample(qs, 200)
p10QuerySetMean=0
cnt=1
docsFnd=0
for ele in qs:
    ele=parse_text(ele)
    sotArr=getSOTResults(ele)
    if(len(sotArr)>0):
        p10QuerySetMean+=evalP10Score(getNeuralOp(ele),sotArr,BUCKET_SIZE)
        docsFnd+=1
    cnt+=1
p10QuerySetMean=p10QuerySetMean/docsFnd

What makes a good life? Lessons from the longest study on happiness | Robert Waldinger
Glenn Greenwald: Why privacy matters
The future we&#39;re building -- and boring | Elon Musk
How to speak so that people want to listen | Julian Treasure
4 reasons to learn a new language | John McWhorter
Richard Ledgett: The NSA responds to Edward Snowden&#39;s TED Talk
What will future jobs look like? | Andrew McAfee
How the blockchain is changing money and business | Don Tapscott
The difference between being &quot;not racist&quot; and antiracist | Ibram X. Kendi
The thrilling potential of SixthSense technology | Pranav Mistry
Sarah Kaminsky: My father the forger
David Logan: Tribal leadership
Jeremy Gilley: One day of peace
4 ways to make a city more walkable | Jeff Speck
Naif Al-Mutawa: Superheroes inspired by Islam
Paul Zak: Trust, morality - and oxytocin
A memory scientist's advice on reporting harassment and discrimination | Julia Shaw
Scientists must be free to learn, to speak and to challeng

Capitalism isn't an ideology -- it's an operating system | Bhu Srinivasan
Bill Clinton: TED Prize wish: Let's build a health care system in Rwanda
Why are drug prices so high? Investigating the outdated US patent system | Priti Krishtel
How a geospatial nervous system could help us design a better future | Jack Dangermond
The world's first crowdsourced space traffic monitoring system | Moriba Jah
Damon Horowitz calls for a "moral operating system"
What the US health care system assumes about you | Mitchell Katz
Rebecca Onie: What if our healthcare system kept us healthy?
An election system that puts voters (not politicians) first | Amber McReynolds
Philip Howard: Four ways to fix a broken legal system
Hackers: the internet's immune system | Keren Elazari
How forgiveness can create a more just legal system | Martha Minow
What's needed to bring the US voting system into the 21st century | Tiana Epps-Johnson
We the People, and the Republic we must reclaim | Lawrence Lessig
Our refugee sys

3 thoughtful ways to conserve water | Lana Mazahreh
A young scientist's quest for clean water | Deepika Kurup
Anupam Mishra: The ancient ingenuity of water harvesting
Garth Lenz: The true cost of oil
How we can make crops survive without water | Jill Farrant
Sonaar Luthra: Meet the Water Canary
Fahad Al-Attiya: A country with no water
Michael Pritchard: How to make filthy water drinkable
The cost of workplace stress -- and how to reduce it | Rob Cooke
Mechai Viravaidya: How Mr. Condom made Thailand a better place
R.A. Mashelkar: Breakthrough designs for ultra-low-cost products
Ludwick Marishane: A bath without water
Romulus Whitaker: The real danger lurking in the water
Racism has a cost for everyone | Heather C. McGhee
Why you don't need 8 glasses of water a day | Body Stuff with Dr. Jen Gunter
Thulasiraj Ravilla: How low-cost eye care can be world-class
A precise, three-word address for every place on earth | Chris Sheldrick
Benjamin Wallace: The price of happiness
Hunting for dinosa

How to grow a forest in your backyard | Shubhendu Sharma
How community-led conservation can save wildlife | Moreangels Mbizah
Thoughts on humanity, fame and love | Shah Rukh Khan
Shubhendu Sharma: How to grow a tiny forest anywhere
The global movement to restore nature&#39;s biodiversity | Thomas Crowther
Why are these 32 symbols found in caves all over Europe | Genevieve von Petzinger
Magical houses, made of bamboo | Elora Hardy
New York -- before the City | Eric Sanderson
How animals and plants are evolving in cities | Menno Schilthuizen
The danger of a single story | Chimamanda Ngozi Adichie
Let&#39;s make the world wild again | Kristine Tompkins
Four billion years of evolution in six minutes | Prosanta Chakrabarty
How trees talk to each other | Suzanne Simard
100 solutions to reverse global warming | Chad Frischmann
Garth Lenz: The true cost of oil
What it&#39;s like to be a Muslim in America | Dalia Mogahed
Cradle to cradle design | William McDonough
A taboo-free way to talk about

The history of our world in 18 minutes | David Christian
You can grow new brain cells. Here&#39;s how | Sandrine Thuret
Is the world getting better or worse? A look at the numbers | Steven Pinker
The brain benefits of deep sleep -- and how to get more of it |  Dan Gartenberg
How the pandemic will shape the near future | Bill Gates
Sleep is your superpower | Matt Walker
Will automation take away all our jobs? | David Autor
How will we survive when the population hits 10 billion? | Charles C. Mann
The puzzle of motivation | Dan Pink
Can we create new senses for humans? | David Eagleman
The astounding athletic power of quadcopters | Raffaello D&#39;Andrea
How trees talk to each other | Suzanne Simard
How to build your confidence -- and spark it in others | Brittany Packnett
Do you really know why you do what you do? | Petter Johansson
A kinder, gentler philosophy of success | Alain de Botton
Mark Kendall: Demo: A needle-free vaccine patch that&#39;s safer and way cheaper
Should you be abl

Why humans run the world | Yuval Noah Harari
The tyranny of merit | Michael Sandel
What makes a good life? Lessons from the longest study on happiness | Robert Waldinger
Atheism 2.0 | Alain de Botton
A Darwinian theory of beauty | Denis Dutton
Capitalism will eat democracy -- unless we speak up | Yanis Varoufakis
How your brain decides what is beautiful | Anjan Chatterjee
How great leaders inspire action | Simon Sinek
The unheard story of David and Goliath | Malcolm Gladwell
The genius behind some of the world&#39;s most famous buildings | Renzo Piano
The magic of Fibonacci numbers | Arthur Benjamin
How to fix a broken heart | Guy Winch
What are the most important moral problems of our time? | Will MacAskill
The habits of happiness | Matthieu Ricard
Mentalism, mind reading and the art of getting inside your head | Derren Brown
The surprising science of happiness | Dan Gilbert
How Quantum Biology Might Explain Life’s Biggest Questions | Jim Al-Khalili | TED Talks
Why good leaders make y

The danger of a single story | Chimamanda Ngozi Adichie
What we're missing in the debate about immigration | Duarte Geraldino
Why Brexit happened -- and what to do next | Alexander Betts
Our immigration conversation is broken -- here's how to have a better one | Paul A. Kramer
What's really happening at the US-Mexico border -- and how we can do better | Erika Pinheiro
Facebook's role in Brexit — and the threat to democracy | Carole Cadwalladr
Billy Graham: Technology, faith and human shortcomings
How the Panama Papers journalists broke the biggest leak in history | Gerard Ryle
Pankaj Ghemawat: Actually, the world isn't flat
The psychology of your future self | Dan Gilbert
Tan Le: My immigration story
It's time to reclaim religion | Sharon Brous
Our refugee system is failing. Here's how we can fix it | Alexander Betts
My Daughter, Malala | Ziauddin Yousafzai | TED Talks
How guest worker visas could transform the US immigration system | David J. Bier
The US is addicted to incarceration. 

In [15]:
print("P10 Robust-05 score: " + str(p10QuerySetMean))

P10 Robust-05 score: 0.3649122807017543


In [8]:
#MS-marco AP
filepath = "query/rob05-fin.txt"
qs=[] #query set
with open(filepath,encoding="utf8") as fp:
    for cnt, line in enumerate(fp):
        qs.append(line)
qs = random.sample(qs, 200)
apQuerySetMean=0
cnt=1
docsFnd=0
for ele in qs:
    ele=parse_text(ele)
    sotArr=getSOTResults(ele)
    if(len(sotArr)>0):
        apQuerySetMean+=evalP10Score(getNeuralOp(ele),sotArr,BUCKET_SIZE)
        docsFnd+=1
    cnt+=1
apQuerySetMean=apQuerySetMean/docsFnd

See how the rest of the world lives, organized by income | Anna Rosling Rönnlund
5 ways to lead in an era of constant change | Jim Hemerling
How to gain control of your free time | Laura Vanderkam
This is what makes employees happy at work | The Way We Work, a TED series
Yves Morieux: As work gets more complex, 6 rules to simplify
How to build a company where the best ideas win | Ray Dalio
8 lessons on building a company people enjoy working for | The Way We Work, a TED series
The way we think about charity is dead wrong | Dan Pallotta
How to design a library that makes kids want to read |  Michael Bierut
The single biggest reason why start-ups succeed | Bill Gross
Why humans run the world | Yuval Noah Harari
A lesson on looking | Amy Herman
What Islam really says about women | Alaa Murabit
How fake handbags fund terrorism and organized crime | Alastair Gray
Ricardo Semler: Radical wisdom for a company, a school, a life
The future of psychedelic-assisted psychotherapy | Rick Doblin
A y

Why the secret to success is setting the right goals | John Doerr
Why gender-based marketing is bad for business | Gaby Barrios
Nirmalya Kumar: India&#39;s invisible innovation
The dirty secret of capitalism -- and a new way forward | Nick Hanauer
Why you should define your fears instead of your goals | Tim Ferriss
The way we think about charity is dead wrong | Dan Pallotta
All it takes is 10 mindful minutes | Andy Puddicombe
How online marketplaces can help local economies, not hurt them | Amane Dannouni
My journey from Marine to actor | Adam Driver
Why domestic violence victims don&#39;t leave | Leslie Morgan Steiner
Your body language may shape who you are | Amy Cuddy
Stefan Sagmeister: Happiness by design
Marc Pachter: The art of the interview
Inside the massive (and unregulated) world of surveillance tech | Sharon Weinberger
Looking for a job? Highlight your ability, not your experience | Jason Shen
The way we think about charity is dead wrong | Dan Pallotta
Why gender-based marke

The best stats you&#39;ve ever seen | Hans Rosling
Math class needs a makeover | Dan Meyer
How America&#39;s public schools keep kids in poverty | Kandice Sumner
How we can store digital data in DNA | Dina Zielinski
Thomas Goetz: It's time to redesign medical data
JP Rangaswami: Information is food
Frederic Kaplan: How I built an information time machine
How bad data keeps us from good AI | Mainak Mazumdar
Why you should get paid for your data | Jennifer Zhu Scott
Kenneth Cukier: Big data is better data
How data is helping us unravel the mysteries of the brain | Steve McCarroll
Philip Evans: How data will transform business
John Wilbanks: Let's pool our medical data
Your company's data could help end world hunger | Mallory Freeman
Tim Berners-Lee: The next Web of open, linked data
The beauty of data visualization | David McCandless
The human insights missing from big data | Tricia Wang
Wireless data from every light bulb | Harald Haas
Insightful human portraits made from data | R. Luke

Why teens confess to crimes they didn't commit | Lindsay Malloy
Community-powered criminal justice reform | Raj Jayadev
Taryn Simon photographs secret sites
What happened when we tested thousands of abandoned rape kits in Detroit | Kym Worthy
Inside the mind of a former radical jihadist | Manwar Ali
A memory scientist's advice on reporting harassment and discrimination | Julia Shaw
How the West can adapt to a rising Asia | Kishore Mahbubani
How I help free innocent people from prison | Ronald Sullivan
His Holiness Pope Francis | Our moral imperative to act on climate change [Italian]
Chimps have feelings and thoughts. They should also have rights
Help for kids the education system ignores | Victor Rios
Ben Cameron: The true power of the performing arts
Why women should tell the stories of humanity | Jude Kelly
His Holiness Pope Francis | Why the only future worth building includes everyone | TED Talks
How India's smartphone revolution is creating a new generation of readers and writers

A smart new business loan for people with no credit | Shivani Siroya
How to build a business that lasts 100 years | Martin Reeves
Management lessons from Chinese business and philosophy | Fang Ruan
Why it&#39;s too hard to start a business in Africa -- and how to change it | Magatte Wade
What really motivates people to be honest in business | Alexander Wagner
How to build a company where the best ideas win | Ray Dalio
Philip Evans: How data will transform business
Why working from home is good for business | The Way We Work, a TED series
Ray Anderson: The business logic of sustainability
Why being respectful to your coworkers is good for business | Christine Porath
The business benefits of doing good | Wendy Woods
7 tools for building a business people trust | Marcos Aguiar
Why gender-based marketing is bad for business | Gaby Barrios
The future we&#39;re building -- and boring | Elon Musk
4 myths and misunderstandings about doing business in Africa | Nomava Zanazo
How great leaders in

How yarn bombing grew into a worldwide movement | Magda Sayeg
Malcolm Gladwell: The strange tale of the Norden bombsight
Sharmeen Obaid Chinoy: Inside a school for suicide bombers
A Saudi, an Indian and an Iranian walk into a Qatari bar ... | Maz Jobrani
Why &quot;biofabrication&quot; is the next industrial revolution | Suzanne Lee
You aren&#39;t at the mercy of your emotions -- your brain creates them | Lisa Feldman Barrett
New bionics let us run, climb and dance | Hugh Herr
Mark Bezos: A life lesson from a volunteer firefighter
Suzanne Lee: Grow your own clothes
Did you hear the one about the Iranian-American? | Maz Jobrani
Simple hacks for life with Parkinson&#39;s | Mileha Soneji
Karima Bennoune: The side of terrorism that doesn&#39;t make headlines
The moral dangers of non-lethal weapons | Stephen Coleman
How fake handbags fund terrorism and organized crime | Alastair Gray
Trevor Aaronson: How this FBI strategy is actually creating US-based terrorists
Jehane Noujaim: TEDPrize wish

In [9]:
print("AP Robust-05 score: " + str(apQuerySetMean))

AP Robust-05 score: 0.4407407407407406
